# Importing all the necessary modules

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np


# Downloading the CIFAR10 dataset

## The CIFAR-10 dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.




In [2]:
def download_CIFAR10_dataset():
    """Downloads the CIFAR-10 dataset in the data folder and returns training data and testing data."""
    train_data = datasets.CIFAR10(
        root = "data",
        download=True,
        train=True,
        transform=ToTensor()
    )
    validation_data = datasets.CIFAR10(
        root = "data",
        download=True,
        train=False,
        transform=ToTensor()
    ) 

    return train_data,validation_data

In [3]:
train_data, validation_data = download_CIFAR10_dataset()
print(train_data,validation_data)

100.0%


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor() Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


# Making DataLoader Wraper for train data to load data in batches

In [4]:
BATCH_SIZE = 9

train_data_loader = DataLoader(train_data,batch_size=BATCH_SIZE)

# Visualizing the CIFAR-10 dataset

In [5]:
dataiter = iter(train_data_loader)
inputs,labels = next(dataiter)

In [6]:
inputs.shape

torch.Size([9, 3, 32, 32])

In [7]:
images = inputs.numpy()